In [1]:
from google.colab import files

# Step 1: Upload the zip file
uploaded = files.upload()

Saving chess.zip to chess.zip


In [2]:
import zipfile
import os

# Step 2: Unzip the uploaded file
with zipfile.ZipFile('chess.zip', 'r') as zip_ref:
    zip_ref.extractall('chess')

# Step 3: List the files in the directory to confirm
print(os.listdir('chess'))

['chess']


In [3]:
#List the files in the 'cameracal' directory to confirm the contents
print(os.listdir('chess/chess'))


['calibration2.jpg', 'calibration1.jpg', 'calibration4.jpg', 'calibration5.jpg', 'calibration12.jpg', 'calibration19.jpg', 'calibration13.jpg', 'calibration17.jpg', 'calibration14.jpg', 'calibration13 (1).jpg', 'calibration11.jpg', 'calibration16.jpg', 'calibration20.jpg', 'calibration10.jpg', 'calibration6.jpg', 'calibration9.jpg', 'calibration8.jpg', 'calibration7.jpg', 'calibration3.jpg', 'calibration15.jpg', 'calibration18.jpg']


In [4]:
# Install necessary library
'''!pip install moviepy

import zipfile
import os
import numpy as np
import cv2
import glob
import matplotlib.image as mpimg
from moviepy.editor import VideoFileClip

class CameraCalibration:
    def __init__(self, calibration_images_dir, nx, ny):
        """Initialize the camera calibration process."""
        self.objpoints = []  # 3D points in real-world space
        self.imgpoints = []  # 2D points in image plane
        self.camera_matrix = None
        self.dist_coeffs = None

        # Prepare object points
        objp = np.zeros((ny * nx, 3), np.float32)
        objp[:, :2] = np.mgrid[0:nx, 0:ny].T.reshape(-1, 2)

        # Load calibration images
        images = glob.glob(f'{calibration_images_dir}/*.jpg')

        for fname in images:
            img = cv2.imread(fname)
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            ret, corners = cv2.findChessboardCorners(gray, (nx, ny), None)

            if ret:
                self.objpoints.append(objp)
                self.imgpoints.append(corners)

        # Check if any chessboard corners were found
        if self.objpoints and self.imgpoints:
            ret, self.camera_matrix, self.dist_coeffs, _, _ = cv2.calibrateCamera(self.objpoints, self.imgpoints, gray.shape[::-1], None, None)
        else:
            raise ValueError("No calibration images found or no chessboard corners detected.")

    def undistort(self, img):
        """Undistort the input image using the camera matrix and distortion coefficients."""
        if self.camera_matrix is not None and self.dist_coeffs is not None:
            return cv2.undistort(img, self.camera_matrix, self.dist_coeffs, None, self.camera_matrix)
        else:
            raise ValueError("Camera matrix and distortion coefficients are not available.")

class Thresholding:
    def forward(self, img):
        """Apply thresholding to the input image."""
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        _, binary = cv2.threshold(gray, 160, 255, cv2.THRESH_BINARY)
        return np.dstack((binary, binary, binary)) * 255

class PerspectiveTransformation:
    def __init__(self):
        # Define source and destination points for transformation.
        self.src = np.float32([[100, 720], [1200, 720], [750, 480], [530, 480]])
        self.dst = np.float32([[200, 720], [1080, 720], [1080, 0], [200, 0]])

    def forward(self, img):
        """Warp the image to top-down view."""
        img_size = (img.shape[1], img.shape[0])
        M = cv2.getPerspectiveTransform(self.src, self.dst)
        warped = cv2.warpPerspective(img, M, img_size)
        return warped

    def backward(self, img):
        """Warp the image back to original perspective."""
        img_size = (img.shape[1], img.shape[0])
        Minv = cv2.getPerspectiveTransform(self.dst, self.src)
        unwarped = cv2.warpPerspective(img, Minv, img_size)
        return unwarped

class LaneLines:
    def forward(self, img):
        """Detect lane lines and create overlay image."""
        # Dummy lane detection, replace with real lane line detection logic
        lane_overlay = np.zeros_like(img)
        cv2.line(lane_overlay, (300, 700), (400, 400), (0, 255, 0), 10)
        cv2.line(lane_overlay, (900, 700), (900, 400), (0, 255, 0), 10)
        return lane_overlay

    def plot(self, img):
        """Draw detected lane lines on the image."""
        cv2.putText(img, "Lane Detected", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
        return img

class FindLaneLines:
    def __init__(self, nx=9, ny=6):
        """Init the pipeline."""
        self.calibration = CameraCalibration('chess/chess', nx, ny)  # Update path as needed
        self.thresholding = Thresholding()
        self.transform = PerspectiveTransformation()
        self.lanelines = LaneLines()

    def forward(self, img):
        """Process a single frame for lane line detection."""
        original_img = np.copy(img)

        # Step 1: Undistort the image
        img = self.calibration.undistort(img)

        # Step 2: Apply perspective transformation
        img = self.transform.forward(img)

        # Step 3: Apply thresholding to detect lane-like regions
        img = self.thresholding.forward(img)

        # Step 4: Detect lane lines and create an overlay image
        lane_overlay = self.lanelines.forward(img)

        # Step 5: Warp back to original perspective
        lane_overlay = self.transform.backward(lane_overlay)

        # Step 6: Combine original image with lane overlay
        output_img = cv2.addWeighted(original_img, 1, lane_overlay, 0.6, 0)

        # Step 7: Plot additional information (if any)
        output_img = self.lanelines.plot(output_img)

        return output_img

    def process_image(self, input_path, output_path):
        """Process a single image."""
        img = mpimg.imread(input_path)
        out_img = self.forward(img)
        mpimg.imsave(output_path, out_img)

    def process_video(self, input_path, output_path):
        """Process a video frame by frame."""
        clip = VideoFileClip(input_path)
        out_clip = clip.fl_image(self.forward)
        out_clip.write_videofile(output_path, audio=False)

def main():
    findLaneLines = FindLaneLines()
    findLaneLines.process_video("/content/challenge_video.mp4", "output.mp4")

if __name__ == "__main__":
    main()'''


Moviepy - Building video output.mp4.
Moviepy - Writing video output.mp4



t: 100%|██████████| 485/485 [01:07<00:00,  8.67it/s, now=None]WARNING:py.warnings:/usr/local/lib/python3.10/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file /content/challenge_video.mp4, 2764800 bytes wanted but 0 bytes read,at frame 484/485, at time 16.15/16.15 sec. Using the last valid frame instead.
  warnings.warn("Warning: in file %s, "%(self.filename)+



Moviepy - Done !
Moviepy - video ready output.mp4


In [5]:
# Install necessary library
!pip install moviepy

import zipfile
import os
import numpy as np
import cv2
import glob
import matplotlib.image as mpimg
from moviepy.editor import VideoFileClip

class CameraCalibration:
    def __init__(self, calibration_images_dir, nx, ny):
        """Initialize the camera calibration process."""
        self.objpoints = []  # 3D points in real-world space
        self.imgpoints = []  # 2D points in image plane
        self.camera_matrix = None
        self.dist_coeffs = None

        # Prepare object points
        objp = np.zeros((ny * nx, 3), np.float32)
        objp[:, :2] = np.mgrid[0:nx, 0:ny].T.reshape(-1, 2)

        # Load calibration images
        images = glob.glob(f'{calibration_images_dir}/*.jpg')

        for fname in images:
            img = cv2.imread(fname)
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            ret, corners = cv2.findChessboardCorners(gray, (nx, ny), None)

            if ret:
                self.objpoints.append(objp)
                self.imgpoints.append(corners)

        # Check if any chessboard corners were found
        if self.objpoints and self.imgpoints:
            ret, self.camera_matrix, self.dist_coeffs, _, _ = cv2.calibrateCamera(self.objpoints, self.imgpoints, gray.shape[::-1], None, None)
        else:
            raise ValueError("No calibration images found or no chessboard corners detected.")

    def undistort(self, img):
        """Undistort the input image using the camera matrix and distortion coefficients."""
        if self.camera_matrix is not None and self.dist_coeffs is not None:
            return cv2.undistort(img, self.camera_matrix, self.dist_coeffs, None, self.camera_matrix)
        else:
            raise ValueError("Camera matrix and distortion coefficients are not available.")

class Thresholding:
    def forward(self, img):
        """Apply thresholding to the input image."""
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        _, binary = cv2.threshold(gray, 160, 255, cv2.THRESH_BINARY)
        return np.dstack((binary, binary, binary)) * 255

class PerspectiveTransformation:
    def __init__(self):
        # Define source and destination points for transformation.
        self.src = np.float32([[100, 720], [1200, 720], [750, 480], [530, 480]])
        self.dst = np.float32([[200, 720], [1080, 720], [1080, 0], [200, 0]])

    def forward(self, img):
        """Warp the image to top-down view."""
        img_size = (img.shape[1], img.shape[0])
        M = cv2.getPerspectiveTransform(self.src, self.dst)
        warped = cv2.warpPerspective(img, M, img_size)
        return warped

    def backward(self, img):
        """Warp the image back to original perspective."""
        img_size = (img.shape[1], img.shape[0])
        Minv = cv2.getPerspectiveTransform(self.dst, self.src)
        unwarped = cv2.warpPerspective(img, Minv, img_size)
        return unwarped

class LaneLines:
    def forward(self, img):
        """Detect lane lines and create overlay image."""
        lane_overlay = np.zeros_like(img)  # Create an empty overlay with the same shape as the input image
        # Define lane line positions (Dummy positions, replace with actual detection)
        cv2.fillPoly(lane_overlay, [np.array([[300, 720], [400, 400], [400, 720]], np.int32)], (0, 255, 0))  # Left lane
        cv2.fillPoly(lane_overlay, [np.array([[900, 720], [900, 400], [1000, 720]], np.int32)], (0, 255, 0))  # Right lane
        return lane_overlay

    def plot(self, img):
        """Draw detected lane lines on the image."""
        cv2.putText(img, "Lane Detected", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
        return img

class FindLaneLines:
    def __init__(self, nx=9, ny=6):
        """Init the pipeline."""
        self.calibration = CameraCalibration('chess/chess', nx, ny)  # Update path as needed
        self.thresholding = Thresholding()
        self.transform = PerspectiveTransformation()
        self.lanelines = LaneLines()

    def forward(self, img):
        """Process a single frame for lane line detection."""
        original_img = np.copy(img)

        # Step 1: Undistort the image
        img = self.calibration.undistort(img)

        # Step 2: Apply perspective transformation
        img = self.transform.forward(img)

        # Step 3: Apply thresholding to detect lane-like regions
        img = self.thresholding.forward(img)

        # Step 4: Detect lane lines and create an overlay image
        lane_overlay = self.lanelines.forward(img)

        # Step 5: Warp back to original perspective
        lane_overlay = self.transform.backward(lane_overlay)

        # Step 6: Combine original image with lane overlay
        output_img = cv2.addWeighted(original_img, 1, lane_overlay, 0.6, 0)

        # Step 7: Plot additional information (if any)
        output_img = self.lanelines.plot(output_img)

        return output_img

    def process_image(self, input_path, output_path):
        """Process a single image."""
        img = mpimg.imread(input_path)
        out_img = self.forward(img)
        mpimg.imsave(output_path, out_img)

    def process_video(self, input_path, output_path):
        """Process a video frame by frame."""
        clip = VideoFileClip(input_path)
        out_clip = clip.fl_image(self.forward)
        out_clip.write_videofile(output_path, audio=False)

def main():
    findLaneLines = FindLaneLines()
    findLaneLines.process_video("/content/challenge_video.mp4", "output.mp4")

if __name__ == "__main__":
    main()


Moviepy - Building video output.mp4.
Moviepy - Writing video output.mp4



Moviepy - Done !
Moviepy - video ready output.mp4
